# **Data**

In [ ]:
!pip install datasketch
!pip install xxhash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string
# from datasketch import MinHash, LeanMinHash
# import xxhash
from tqdm import tqdm

In [ ]:
data = pd.read_csv("tweets_v172_final.csv")

## **remove duplicates & near duplicates**

We find near-duplicates by hashing the texts of tweets after lowercasing and stripping punctuation. Hashing is performed using MinHash (Broder, 1997), with 16 permutations.

In [ ]:
data = data.drop_duplicates(subset=['text'], keep='first')

In [ ]:
def hash_tweet(tweet, num_perm=16):
    def normalize_text(text):
        text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
        text = text.lower()
        return text
    def minhash(seq):
        # https://skeptric.com/minhash/
        m = MinHash(num_perm=num_perm, hashfunc=xxhash.xxh64_intdigest)
        for s in seq:
            m.update(s.encode('utf8'))
        return LeanMinHash(m)
    tokens = normalize_text(tweet).split()  # whitespace tokenization
    return minhash(tokens)
written_hashes = set()
data['duplicate'] = False
for index in tqdm(range(len(data))):
    tweet_hash = hash_tweet(data['text'].iloc[index])
    if tweet_hash in written_hashes:
        data['duplicate'].iloc[index] = True
    else: data['duplicate'].iloc[index] = False
    if not data['duplicate'].iloc[index]:
        written_hashes.add(tweet_hash)

  0%|          | 0/19132 [00:00<?, ?it/s]<ipython-input-7-35c6a91f2421>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: data['duplicate'].iloc[index] = False
  1%|          | 158/19132 [00:00<00:24, 773.43it/s]<ipython-input-7-35c6a91f2421>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: data['duplicate'].iloc[index] = False
<ipython-input-7-35c6a91f2421>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['duplicate

In [ ]:
data = data[~data.duplicate]
data = data.drop(['duplicate'], axis=1)

In [ ]:
data.to_csv('../content/dat.csv', index=False)